In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostClassifier
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier

In [3]:
train = pd.read_csv('../Train/train_set_binary_encoding.csv')

train.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,position_b1,position_b0,plan_configuration_b3,plan_configuration_b2,plan_configuration_b1,plan_configuration_b0,legal_ownership_status_b2,legal_ownership_status_b1,legal_ownership_status_b0,damage_grade
0,6,487,12198,2,30,6,5,1,1,0,...,1,0,0,0,0,1,0,0,1,3
1,8,900,2812,2,10,8,7,0,1,0,...,0,1,0,0,0,1,0,0,1,2
2,21,363,8973,2,10,5,5,0,1,0,...,1,0,0,0,0,1,0,0,1,3
3,22,418,10694,2,10,6,5,0,1,0,...,0,1,0,0,0,1,0,0,1,2
4,11,131,1488,3,30,8,9,1,0,0,...,0,1,0,0,0,1,0,0,1,3


In [4]:
X, y = train.iloc[:,:-1], train.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

In [11]:
estimators = []

estimators.append(("rf_1", RandomForestClassifier(n_estimators= 270, n_jobs=-1,
                                                random_state=300, min_samples_leaf= 2,
                                                max_features = 23, max_leaf_nodes = 10000)))
estimators.append(("rf_2", RandomForestClassifier(n_estimators= 289, n_jobs=-1,
                                                  random_state=300, min_samples_leaf= 3,
                                                  max_features = 34, max_leaf_nodes = 10000)))
estimators.append(("cat", CatBoostClassifier(silent=True)))
estimators.append(("lgbm", lgb.LGBMClassifier(force_col_wise=False, force_row_wise=False, learning_rate=0.5,
               max_depth=20, n_estimators=40, num_leaves=30,
               objective='multiclass', random_state=123)))
estimators.append(("xgb", xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.4, max_delta_step=0, max_depth=14,
              min_child_weight=1, missing=np.nan, monotone_constraints='()',
              n_estimators=11, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=123, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)))

In [12]:
model = VotingClassifier(estimators=estimators, voting='soft', weights=(4, 3, 2, 1, 1))
model.fit(X_train, y_train)

/home/ben/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:00:47] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


VotingClassifier(estimators=[('rf_1',
                              RandomForestClassifier(max_features=23,
                                                     max_leaf_nodes=10000,
                                                     min_samples_leaf=2,
                                                     n_estimators=270,
                                                     n_jobs=-1,
                                                     random_state=300)),
                             ('rf_2',
                              RandomForestClassifier(max_features=34,
                                                     max_leaf_nodes=10000,
                                                     min_samples_leaf=3,
                                                     n_estimators=289,
                                                     n_jobs=-1,
                                                     random_state=300)),
                             ('cat',
                              <catb

In [13]:
preds = model.predict(X_test)
f1 = f1_score(y_test, preds, average='micro')
print("F1 Score", f1)

F1 Score 0.739303940754384


In [14]:
estimators = []

estimators.append(("rf_1", RandomForestClassifier(n_estimators= 270, n_jobs=-1,
                                                random_state=300, min_samples_leaf= 2,
                                                max_features = 23, max_leaf_nodes = 10000)))
estimators.append(("rf_2", RandomForestClassifier(n_estimators= 289, n_jobs=-1,
                                                  random_state=300, min_samples_leaf= 3,
                                                  max_features = 34, max_leaf_nodes = 10000)))
estimators.append(("cat", CatBoostClassifier(silent=True)))

In [15]:
model = VotingClassifier(estimators=estimators, voting='soft', weights=(3, 2, 1))
model.fit(X_train, y_train)

VotingClassifier(estimators=[('rf_1',
                              RandomForestClassifier(max_features=23,
                                                     max_leaf_nodes=10000,
                                                     min_samples_leaf=2,
                                                     n_estimators=270,
                                                     n_jobs=-1,
                                                     random_state=300)),
                             ('rf_2',
                              RandomForestClassifier(max_features=34,
                                                     max_leaf_nodes=10000,
                                                     min_samples_leaf=3,
                                                     n_estimators=289,
                                                     n_jobs=-1,
                                                     random_state=300)),
                             ('cat',
                              <catb

In [16]:
preds = model.predict(X_test)
f1 = f1_score(y_test, preds, average='micro')
print("F1 Score", f1)

F1 Score 0.7398795134492153
